In [4]:
import sys
from os.path import abspath
sys.path.insert(0, abspath('..'))

from os.path import join

import scipy
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from spherecluster import VonMisesFisherMixture

import torch
from torchSTC.data import load_data
from torchSTC.modules import STC
from torchSTC.metrics import SpacePlot, Evaluate
#from torchSTC.utils import SphericalKMeans


plot = SpacePlot()
eval = Evaluate()

ImportError: cannot import name '_tolerance' from 'spherecluster.regacy' (/home/godwin/Documents/academic/PPD/torchSTC/stc2/lib/python3.11/site-packages/spherecluster/regacy.py)

In [2]:
cur = abspath("")
dataset = 'Biomedical'
data_in_dir=join(cur, "../..", "datasets")
dataset_dir=join(data_in_dir, dataset)

In [3]:
checkpoint_dir=join(dataset_dir, 
                    "artefacts",
                    "STC-d48:500:500:2000:20-epoch15-datBiomedical-wdeWord2Vec-scaMinMax-tfeSIF-normNone-initKmeans"
                    )

checkpoint = "STC-datBiomedical-wdeWord2Vec-scaMinMax-tfeSIF-normNone-initKmeans.pth"
checkpoint_path = join(checkpoint_dir, checkpoint)
checkpoint_path

'/home/godwin/Documents/academic/PPD/torchSTC/demos/Biomedical/../../datasets/Biomedical/artefacts/STC-d48:500:500:2000:20-epoch15-datBiomedical-wdeWord2Vec-scaMinMax-tfeSIF-normNone-initKmeans/STC-datBiomedical-wdeWord2Vec-scaMinMax-tfeSIF-normNone-initKmeans.pth'

In [4]:
x, y = load_data(dataset=dataset_dir, word_emb='Word2Vec', transform='SIF', scaler='MinMax', norm=None)
n_clusters = len(torch.unique(torch.tensor(y)))

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=0)

# conversion des données en tenseurs
X_train = torch.tensor(X_train, dtype=torch.float)
X_test = torch.tensor(X_test, dtype=torch.float)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

stc = STC(hidden_dims=[torch.Tensor(X_train).shape[-1], 500, 500, 2000, 20], n_clusters=n_clusters)
stc.from_pretrained(checkpoint_path)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

### Embedding started...
Test w2v ==> ==> ==> 
#### SIF embedding started...
SIF-Embedding 19999 documents with 48-dimensional word vectors...
SVD decomposition...
### SIF embedding completed...
### Embedding completed...
[embed_docs] XX shape:  (20000, 48)
MinMax scaling completed...
No normalization applied...


(torch.Size([18000, 48]),
 torch.Size([2000, 48]),
 torch.Size([18000]),
 torch.Size([2000]))

In [5]:
z2 = stc.autoencoder.encoder(X_train)

In [ ]:
# comprehension list with 5 runs of kmeans, get average and std of metrics
avg_w2v_mmx_ikm2 = []
tmp = []
for i in range(5):
    kmeans = KMeans(n_clusters=n_clusters, n_init=50)
    kmeans.fit(z2.detach().numpy())
    y_km_pred = kmeans.labels_
    tmp.append(eval.allMetrics(y_train.detach().numpy(), y_km_pred))

avg_w2v_mmx_ikm2 = np.array(tmp)
np.round(avg_w2v_mmx_ikm2.mean(axis=0), 3) * 100, avg_w2v_mmx_ikm2.std(axis=0)

In [ ]:
avg_w2v_mmx_iskm2 = []
tmp = []
for i in range(5):
    skmeans = SphericalKmeans(n_clusters=n_clusters, n_init=50)
    skmeans.fit(z2.detach().numpy())
    y_skm_pred = skmeans.labels_
    tmp.append(eval.allMetrics(y_train.detach().numpy(), y_skm_pred))

avg_w2v_mmx_iskm2 = np.array(tmp)
np.round(avg_w2v_mmx_iskm2.mean(axis=0), 3) * 100, avg_w2v_mmx_iskm2.std(axis=0)

In [ ]:
kmeans = KMeans(n_clusters=n_clusters, n_init=50)
kmeans.fit(z2.detach().numpy())
y_km_pred2 = kmeans.labels_


skmeans = SphericalKmeans(n_clusters=n_clusters, n_init=50)
skmeans.fit(z2.detach().numpy())
y_skm_pred2 = skmeans.labels_

In [ ]:
print(min(y_km_pred2), len(np.unique(y_km_pred2)), max(y_km_pred2))
print(min(y_skm_pred2), len(np.unique(y_skm_pred2)), max(y_skm_pred2))

In [ ]:
eval.allMetrics(y_train.detach().numpy(), y_km_pred2)

In [ ]:
eval.allMetrics(y_train.detach().numpy(), y_skm_pred2)

In [ ]:
plot.commonSpace_plot(z2.detach().numpy(), comp=[0, 1], tagLabels=y_km_pred2,
                      data_name="Snippets hgf final Kmeans assign", dimred = 'UMAP')

In [ ]:
plot.commonSpace_plot(z2.detach().numpy(), comp=[0, 1], tagLabels=y_km_pred2,
                      data_name="Snippets hgf final Kmeans assign", dimred = 'TSNE')

In [6]:
# movMF-hard
vmf_hard = VonMisesFisherMixture(n_clusters=n_clusters, posterior_type='soft')


In [14]:
x = z2.detach().numpy()

In [15]:
vmf_hard.fit(x)

# vmf_hard.cluster_centers_
# vmf_hard.labels_
# vmf_hard.weights_
# vmf_hard.concentrations_
# vmf_hard.inertia_

/home/godwin/Documents/academic/PPD/torchSTC/stc2/lib/python3.11/site-packages/spherecluster/von_mises_fisher_mixture.py:58: RuntimeWarning: divide by zero encountered in log
  return np.log(_vmf_normalize(kappa, n_features) * np.exp(kappa * X.dot(mu).T))
/home/godwin/Documents/academic/PPD/torchSTC/stc2/lib/python3.11/site-packages/spherecluster/von_mises_fisher_mixture.py:72: RuntimeWarning: overflow encountered in scalar multiply
  denom = np.power(2.0 * np.pi, dim / 2.0) * iv(dim / 2.0 - 1.0, kappa)


ValueError: VMF scaling denominator was inf.